# Take Home Assignment 1

## Problem Statement
**What qualities of crude oil lead to pipeline accidents?**

Pipelines are generally regarded as a safe way to transport fuel, a far better alternative to tanker trucks or freight trains. But when a pipeline does fail, the consequences can be catastrophic; since 1986, pipeline accidents have killed more than 500 people, injured over 4,000 and cost nearly seven billion dollars in property damage. 

Using any resources available to you, can you determine *what phyisical chemical properties* of crude oil could be responsible for pipeline accidents?

---


### Scoping and Assumptions

To tackle this question, first I will make some qualifying assumption to scope the problem.
- We are going to focus on North America (with a slight preference to Canada)
- We are going to assume all North American crude oil has roughly the same chemistry and physical properties
- more here tbd

### Backgrounder - What is a pipeline accident and why do they happen?

To answer why pipeline accidents happen, first we have to figure out what we mean when we say "accident". We will start by analysing the [pipeline incident dataset](https://open.canada.ca/data/en/dataset/7dffedc4-23fa-440c-a36d-adf5a6cc09f1) provided by the National Energy Board. This will provide us of an idea of what kind of incidents occured during the reporting period (2008-2025). Given that this is a dataset of all pipelines, not just crude oil, we might also get some insight into how or why incidents concerning crude oil are different than other petrochemicals.

In [13]:
!pip install folium

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached MarkupSafe-3.0.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (4.0 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached MarkupSafe-3.0.2-cp313-cp313-macosx_11_0_arm64.whl (12 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium as fm

In [15]:
df = pd.read_csv(filepath_or_buffer= 'data/in/pipeline-incidents-comprehensive-data.csv',encoding='utf-8', encoding_errors='ignore')

In [21]:
df['Substance carried'].value_counts()

Substance carried
Natural Gas                                                                                            466
Crude Oil                                                                                              323
Crude Oil, Crude Oil Sour Heavy, Crude Oil Sour Light, Crude Oil Sweet Heavy, Crude Oil Sweet Light     65
Natural Gas Sweet                                                                                       54
Refined Products-Aviation                                                                               44
Natural Gas, Natural Gas Sweet                                                                          43
Natural Gas Sour                                                                                        39
White Water                                                                                             12
Refined Products-Aviation, Refined Products-Distillate, Refined Products-Gasoline                        8
Natural Gas Sour, N

In [ ]:
#instantiate map and get center
map_center = [df['Latitude'].mean(), df['Longitude'].mean()]
pipeline_map = fm.Map(location=map_center, zoom_start=3.5)

#plot incidents on map
for _, row in df[df['Substance carried']=='Crude Oil'].iterrows():
    fm.Marker([row['Latitude'], row['Longitude']]).add_to(pipeline_map)

#display map
display(pipeline_map)